In [1]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
# instance_id 样本编号
# item_id 广告商品编号
# item_category_list 广告商品的的类目列表 分割; item_property_list_0 item_property_list_1 item_property_list_2
# item_property_list 广告商品的属性列表 分割 1 2 3
# item_brand_id 广告商品的品牌编号
# item_city_id 广告商品的城市编号
# item_price_level 广告商品的价格等级
# item_sales_level 广告商品的销量等级
# item_collected_level 广告商品被收藏次数的等级
# item_pv_level 广告商品被展示次数的等级
# user_id 用户的编号
# 'user_gender_id', 用户的预测性别编号
# 'user_age_level', 用户的预测年龄等级
# 'user_occupation_id', 用户的预测职业编号
# 'user_star_level' 用户的星级编号
# context_id 上下文信息的编号
#  context_timestamp 广告商品的展示时间
# context_page_id 广告商品的展示页面编号
# predict_category_property
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))

def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    del data['context_timestamp_tmp']


    print('predict_category_property_ing_0')
    for i in range(3):
        data['predict_category_%d'%(i)] = data['predict_category_property'].apply(
            lambda x:str(x.split(";")[i]).split(":")[0] if len(x.split(";")) > i else " "
        )

    # print('predict_category_property_ing_1')
    # for i in range(3):
    #     data['predict_property_%d'%(i)] = data['predict_category_property'].apply(
    #         lambda x:str(x.split(";")[i]).split(":")[1] if len(x.split(";")) > i else " "
    #     )
    #
    #     for j in range(3):
    #         data['predict_property_%d_%d' % (i,j)] = data['predict_property_%d'%(i)].apply(
    #             lambda x: x.split(",")[j] if len(x.split(",")) > j else -1
    #         )

    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']

    return data

In [2]:
print('train')
train = pd.read_csv('train.txt',sep=" ")

train = pre_process(train)

all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-21 23:59:59']

train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)



train
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time
predict_category_property_ing_0
all_shape (478138, 35)
2018-09-24 23:59:47
(139586, 35)
(121035, 35)


In [3]:
train

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,...,category_1,category_2,property_0,property_1,property_2,week,hour,predict_category_0,predict_category_1,predict_category_2
78268,4293715837363757697,3412720377098676069,1975590437749032870,3948283326616421003,3,4,4,14,8711326521450559534,0,...,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413,4,1,5799347067982556520,7258015885215914736,7492960463130085436
78269,70764105903083828,285660928590172217,9057103201734987852,548352491538518780,8,9,8,14,8064892827740019258,0,...,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413,4,21,8277336076276184272,5799347067982556520,509660095530134768
78270,3954908518456917085,5202355029344881809,5520678735822176314,548352491538518780,8,9,10,16,1633738185485564030,0,...,5755694407684602296,,2072967855524022579,5131280576272319091,2636395404473730413,4,22,5755694407684602296,836752724084922533,7908382889764677758
78271,4521314384240415059,285660928590172217,9057103201734987852,548352491538518780,8,9,8,14,7983400645978077748,0,...,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413,4,13,8277336076276184272,5799347067982556520,509660095530134768
78272,6005676298575614673,5202355029344881809,5520678735822176314,548352491538518780,8,9,10,16,4831066601005367480,0,...,5755694407684602296,,2072967855524022579,5131280576272319091,2636395404473730413,4,9,5755694407684602296,7447522129235776380,5799347067982556520
78273,655162606126973882,285660928590172217,9057103201734987852,548352491538518780,8,9,8,14,7084099133306871060,1,...,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413,4,18,2011981573061447208,8277336076276184272,5755694407684602296
78274,6388321779120371962,285660928590172217,9057103201734987852,548352491538518780,8,9,8,14,402394192501560149,0,...,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413,4,23,5755694407684602296,8710739180200009128,509660095530134768
78275,2909190494355726720,5202355029344881809,5520678735822176314,548352491538518780,8,9,10,16,6936117071018349386,0,...,5755694407684602296,,2072967855524022579,5131280576272319091,2636395404473730413,4,2,5799347067982556520,5755694407684602296,509660095530134768
78276,8631039197689382476,285660928590172217,9057103201734987852,548352491538518780,8,9,8,14,5881556504781488512,0,...,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413,4,21,2011981573061447208,8277336076276184272,5755694407684602296
78277,1363985136028256226,285660928590172217,9057103201734987852,548352491538518780,8,9,8,14,1944425562356318073,0,...,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413,4,23,2011981573061447208,8277336076276184272,5755694407684602296


In [4]:
print('test')
test_a = pd.read_csv('test.txt',sep=" ")
print(test_a.shape)
test_a = pre_process(test_a)

test
(18371, 26)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time
predict_category_property_ing_0


In [5]:
test_a

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,...,category_1,category_2,property_0,property_1,property_2,week,hour,predict_category_0,predict_category_1,predict_category_2
0,2475218615076601065,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,14,4379490381248358680,0,...,8277336076276184272,,2636395404473730413,7515802706813700848,302719828435032992,1,22,8277336076276184272,5799347067982556520,509660095530134768
1,398316874173557226,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,2629129219935839101,1,...,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,1,16,5755694407684602296,8710739180200009128,7908382889764677758
2,6586402638209028583,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,1092007111346257147,0,...,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,1,14,5755694407684602296,8710739180200009128,7908382889764677758
3,1040996105851528465,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,5353289677517820939,1,...,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,1,15,8710739180200009128,5755694407684602296,7908382889764677758
4,6316278569655873454,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,1098833840660870025,0,...,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,1,15,5755694407684602296,8710739180200009128,2648343924459474923
5,868158305045921978,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,158655638471934821,0,...,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,1,16,8710739180200009128,5755694407684602296,509660095530134768
6,5713520501786699854,1171140458228333440,1547983184598012595,8072963182326625214,5,6,6,15,7254617831443442890,0,...,4879721024980945592,,5131280576272319091,2636395404473730413,3163265386149801264,1,11,4879721024980945592,5799347067982556520,7908382889764677758
7,932945015407923184,7020520887593189887,8324857885829459863,1019055478500227370,7,10,11,14,7588649790698790290,1,...,2011981573061447208,,5131280576272319091,1013419034389836991,5195139481388729954,1,13,2648343924459474923,2011981573061447208,5119332056286574581
8,1919197847086752313,8143258757857432437,8324857885829459863,1019055478500227370,8,10,13,15,7745527540524595604,0,...,2011981573061447208,,2636395404473730413,4621934203383159480,1243768796732904674,1,11,2011981573061447208,7908382889764677758,8277336076276184272
9,304887065966615346,8143258757857432437,8324857885829459863,1019055478500227370,8,10,13,15,2401138934950272514,0,...,2011981573061447208,,2636395404473730413,4621934203383159480,1243768796732904674,1,14,2011981573061447208,8277336076276184272,5755694407684602296


In [6]:
# 这里是增加的内容
import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count


In [7]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)

train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)
y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')



2018-09-20 00:00:01
2018-09-18 00:00:01
1 days 23:59:54
2018-09-25 00:00:02
2018-09-23 00:00:02
1 days 23:59:44
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43


In [8]:
print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data



(18371, 35)


In [9]:
print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
feat_set = list(test_a.columns)
for i,feat in enumerate(feat_set):
    tmp = lb.fit_transform((list(train[feat])+list(val[feat])+list(test_a[feat])))
    print(tmp)
    enc.fit(tmp.reshape(-1,1))
    x_train = enc.transform(lb.transform(train[feat]).reshape(-1, 1))
    x_test = enc.transform(lb.transform(test_a[feat]).reshape(-1, 1))
    x_val = enc.transform(lb.transform(val[feat]).reshape(-1, 1))
    if i == 0:
        X_train, X_test,X_val = x_train, x_test,x_val
    else:
        X_train, X_test,X_val = sparse.hstack((X_train, x_train)), sparse.hstack((X_test, x_test)),sparse.hstack((X_val, x_val))


lr = LogisticRegression()


lr.fit(X_train, y_train)
proba_val = lr.predict_proba(X_val)[:,1]
proba_sub = lr.predict_proba(X_val)[:,1]


baseline ing ... ...
Index(['item_id', 'item_brand_id', 'item_city_id', 'item_price_level',
       'item_sales_level', 'item_collected_level', 'item_pv_level', 'user_id',
       'user_gender_id', 'user_age_level', 'user_occupation_id',
       'user_star_level', 'context_id', 'context_page_id', 'shop_id',
       'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level',
       'shop_score_service', 'shop_score_delivery', 'shop_score_description',
       'category_0', 'category_1', 'category_2', 'property_0', 'property_1',
       'property_2', 'week', 'hour', 'predict_category_0',
       'predict_category_1', 'predict_category_2', 'user_count', 'item_count'],
      dtype='object')
[3269  292 4993 ..., 1772 1772 1772]
[ 408 1858 1158 ..., 1037 1037 1037]
[49  9  9 ..., 49 49 49]
[3 8 8 ..., 6 6 6]
[ 4  9  9 ..., 10 10 10]
[ 4  8 10 ..., 12 12 12]
[14 14 16 ..., 11 11 11]
[118967 110301  22466 ...,  79671  96317  90207]
[1 1 1 ..., 1 1 1]
[1 4 4 ..., 5 6 4]
[4 4 4 ..., 1 1 1]

In [10]:
X_train.shape

(139586, 463518)

In [11]:

import lightgbm as lgb

gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=64,
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.75,
                        # reg_alpha = 0.4
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)

[1]	valid_0's binary_logloss: 0.683887
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.674805
[3]	valid_0's binary_logloss: 0.665905
[4]	valid_0's binary_logloss: 0.657175
[5]	valid_0's binary_logloss: 0.648607
[6]	valid_0's binary_logloss: 0.640215
[7]	valid_0's binary_logloss: 0.631974
[8]	valid_0's binary_logloss: 0.623888
[9]	valid_0's binary_logloss: 0.615952
[10]	valid_0's binary_logloss: 0.608166
[11]	valid_0's binary_logloss: 0.600522
[12]	valid_0's binary_logloss: 0.593024
[13]	valid_0's binary_logloss: 0.58566
[14]	valid_0's binary_logloss: 0.578423
[15]	valid_0's binary_logloss: 0.571316
[16]	valid_0's binary_logloss: 0.564332
[17]	valid_0's binary_logloss: 0.557475
[18]	valid_0's binary_logloss: 0.55074
[19]	valid_0's binary_logloss: 0.544121
[20]	valid_0's binary_logloss: 0.537606
[21]	valid_0's binary_logloss: 0.531198
[22]	valid_0's binary_logloss: 0.524916
[23]	valid_0's binary_logloss: 0.51873
[24]	valid_0's binary_logloss

[208]	valid_0's binary_logloss: 0.122611
[209]	valid_0's binary_logloss: 0.122098
[210]	valid_0's binary_logloss: 0.121596
[211]	valid_0's binary_logloss: 0.121098
[212]	valid_0's binary_logloss: 0.120605
[213]	valid_0's binary_logloss: 0.120121
[214]	valid_0's binary_logloss: 0.119639
[215]	valid_0's binary_logloss: 0.119166
[216]	valid_0's binary_logloss: 0.118699
[217]	valid_0's binary_logloss: 0.118237
[218]	valid_0's binary_logloss: 0.11778
[219]	valid_0's binary_logloss: 0.117331
[220]	valid_0's binary_logloss: 0.116887
[221]	valid_0's binary_logloss: 0.116447
[222]	valid_0's binary_logloss: 0.116014
[223]	valid_0's binary_logloss: 0.115588
[224]	valid_0's binary_logloss: 0.115163
[225]	valid_0's binary_logloss: 0.114741
[226]	valid_0's binary_logloss: 0.114327
[227]	valid_0's binary_logloss: 0.11392
[228]	valid_0's binary_logloss: 0.113519
[229]	valid_0's binary_logloss: 0.113121
[230]	valid_0's binary_logloss: 0.112729
[231]	valid_0's binary_logloss: 0.112344
[232]	valid_0's bi

[413]	valid_0's binary_logloss: 0.0861169
[414]	valid_0's binary_logloss: 0.086087
[415]	valid_0's binary_logloss: 0.0860613
[416]	valid_0's binary_logloss: 0.0860344
[417]	valid_0's binary_logloss: 0.086005
[418]	valid_0's binary_logloss: 0.0859755
[419]	valid_0's binary_logloss: 0.0859477
[420]	valid_0's binary_logloss: 0.0859196
[421]	valid_0's binary_logloss: 0.0858926
[422]	valid_0's binary_logloss: 0.085863
[423]	valid_0's binary_logloss: 0.0858392
[424]	valid_0's binary_logloss: 0.0858121
[425]	valid_0's binary_logloss: 0.0857881
[426]	valid_0's binary_logloss: 0.0857636
[427]	valid_0's binary_logloss: 0.0857368
[428]	valid_0's binary_logloss: 0.0857126
[429]	valid_0's binary_logloss: 0.0856884
[430]	valid_0's binary_logloss: 0.0856631
[431]	valid_0's binary_logloss: 0.0856379
[432]	valid_0's binary_logloss: 0.0856135
[433]	valid_0's binary_logloss: 0.085592
[434]	valid_0's binary_logloss: 0.085571
[435]	valid_0's binary_logloss: 0.0855514
[436]	valid_0's binary_logloss: 0.08552

[609]	valid_0's binary_logloss: 0.0841299
[610]	valid_0's binary_logloss: 0.0841269
[611]	valid_0's binary_logloss: 0.0841244
[612]	valid_0's binary_logloss: 0.0841227
[613]	valid_0's binary_logloss: 0.0841201
[614]	valid_0's binary_logloss: 0.0841165
[615]	valid_0's binary_logloss: 0.0841108
[616]	valid_0's binary_logloss: 0.0841077
[617]	valid_0's binary_logloss: 0.0841021
[618]	valid_0's binary_logloss: 0.0841003
[619]	valid_0's binary_logloss: 0.0840985
[620]	valid_0's binary_logloss: 0.0840966
[621]	valid_0's binary_logloss: 0.0840966
[622]	valid_0's binary_logloss: 0.0840957
[623]	valid_0's binary_logloss: 0.0840935
[624]	valid_0's binary_logloss: 0.0840916
[625]	valid_0's binary_logloss: 0.0840891
[626]	valid_0's binary_logloss: 0.08409
[627]	valid_0's binary_logloss: 0.0840875
[628]	valid_0's binary_logloss: 0.0840854
[629]	valid_0's binary_logloss: 0.0840843
[630]	valid_0's binary_logloss: 0.0840819
[631]	valid_0's binary_logloss: 0.0840792
[632]	valid_0's binary_logloss: 0.08

[808]	valid_0's binary_logloss: 0.0840027
[809]	valid_0's binary_logloss: 0.0840064
[810]	valid_0's binary_logloss: 0.0840045
[811]	valid_0's binary_logloss: 0.0840026
[812]	valid_0's binary_logloss: 0.0840048
[813]	valid_0's binary_logloss: 0.0840046
[814]	valid_0's binary_logloss: 0.0840026
[815]	valid_0's binary_logloss: 0.0840027
[816]	valid_0's binary_logloss: 0.0839984
[817]	valid_0's binary_logloss: 0.0839975
[818]	valid_0's binary_logloss: 0.0839982
[819]	valid_0's binary_logloss: 0.0839997
[820]	valid_0's binary_logloss: 0.0839985
[821]	valid_0's binary_logloss: 0.083996
[822]	valid_0's binary_logloss: 0.0839943
[823]	valid_0's binary_logloss: 0.083991
[824]	valid_0's binary_logloss: 0.0839925
[825]	valid_0's binary_logloss: 0.0839934
[826]	valid_0's binary_logloss: 0.0839959
[827]	valid_0's binary_logloss: 0.0839955
[828]	valid_0's binary_logloss: 0.0839957
[829]	valid_0's binary_logloss: 0.0839966
[830]	valid_0's binary_logloss: 0.0839954
[831]	valid_0's binary_logloss: 0.08

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=64, objective='binary', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=0.75,
       subsample_for_bin=200000, subsample_freq=1)

In [14]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict(X_val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(X_test, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
print(log_loss(y_val,proba_val))
print()
print(log_loss(y_train,gbm.predict(X_train)))
print(log_loss(y_val,y_pred_1))

print(log_loss(y_val,(y_pred_1 + proba_val)/2))

Start predicting...


D:\anaconda\lib\site-packages\lightgbm\basic.py:447: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.0495575471857
0.0858964665614

0.07489126558
0.0839895609694
0.0841199978681


In [15]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier( n_estimators=500,random_state =666, nthread = -1,n_jobs=-1)
model_xgb.fit(X_train, y_train,eval_metric='logloss')


y_xpred_1 = model_xgb.predict_proba(X_val)[:,1]
y_xsub_1 = model_xgb.predict_proba(X_test)[:,1]


In [18]:
print(log_loss(y_train,model_xgb.predict_proba(X_train)))
print(log_loss(y_val, y_xpred_1))

0.0863667298452
0.0837027485755


In [13]:
xx_analyse = pd.DataFrame()
xx_analyse['ture'] = list(y_val)
xx_analyse['pre'] = list(proba_val)
xx_analyse['pre_1'] = list(y_pred_1)
xx_analyse.to_csv('temp.csv',index=False)


sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180322.txt',sep=" ",index=False)

In [ ]:
xx_analyse = pd.DataFrame()
xx_analyse['ture'] = list(y_val)
xx_analyse['pre'] = list(proba_val)
xx_analyse['pre_1'] = list(y_pred_1)
xx_analyse.to_csv('temp.csv',index=False)


sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_xsub_1)
sub.to_csv('20180322.txt',sep=" ",index=False)